In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import wget

In [14]:
training=wget.download('https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip',bar=None)
testing=wget.download('https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip',bar=None)

In [22]:
k=os.getcwd()
print(type(k))

<class 'str'>


In [26]:
zip_ref=zipfile.ZipFile(training,'r')
zip_ref.extractall('D:/win_Downloads/horse-or-humnan')
zip_ref=zipfile.ZipFile(testing,'r')
zip_ref.extractall('D:/win_Downloads/validation-horse-or-humnan')

In [60]:
train_datagen=ImageDataGenerator(rescale=1./255)
validation_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory('D:/win_Downloads/horse-or-humnan',
                                                  target_size=(300,300),
                                                  batch_size=79,
                                                  class_mode='binary'
                                                 )

validation_generator=validation_datagen.flow_from_directory('D:/win_Downloads/validation-horse-or-humnan',
                                                  target_size=(300,300),
                                                  batch_size=32,
                                                  class_mode='binary'
                                                 )

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [70]:
class mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('acc')>0.90):
            print('accuracy reached: 0.999')
            self.model.stop_training=True
callbacks=mycallback()

In [71]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(120,activation='relu'),
    tf.keras.layers.Dense(20,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [72]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 78400)             0         
__________

In [73]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy',
             optimizer=RMSprop(lr=0.001),
             metrics=['acc'])

In [74]:
history=model.fit_generator(train_generator,
                            steps_per_epoch=13,
                            epochs=15,
                            validation_data=validation_generator,
                            validation_steps=8,
                            callbacks=[callbacks],
                            verbose=1
                           )

Epoch 1/15
13/13 [==============================] - 111s 9s/step - loss: 1.6852 - acc: 0.5540 - val_loss: 0.4967 - val_acc: 0.8555
Epoch 2/15
13/13 [==============================] - 121s 9s/step - loss: 0.6677 - acc: 0.6417 - val_loss: 0.5227 - val_acc: 0.7422
Epoch 3/15
13/13 [==============================] - 104s 8s/step - loss: 0.3582 - acc: 0.8734 - val_loss: 0.9326 - val_acc: 0.8164
Epoch 4/15
13/13 [==============================] - 104s 8s/step - loss: 0.6225 - acc: 0.8676 - val_loss: 1.2633 - val_acc: 0.7852
Epoch 5/15
8/8 [==============================] - 13s 2s/step - loss: 0.8878 - acc: 0.8672
accuracy reached: 0.999
13/13 [==============================] - 112s 9s/step - loss: 0.1076 - acc: 0.9542 - val_loss: 0.8878 - val_acc: 0.8672
